In [118]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Flight Delay Analysis") \
    .master("local[*]") \
    .getOrCreate()

In [119]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("hdfs://localhost:9000/user/ketkimaddiwar/airline_data/*/*.csv")

df.show(5)
df.printSchema()
df.count()

25/05/09 14:42:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Reporting_Airline, DOT_ID_Reporting_Airline, IATA_CODE_Reporting_Airline, Tail_Number, Flight_Number_Reporting_Airline, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, OriginWac, DestAirportID, DestAirportSeqID, DestCityMarketID, Dest, DestCityName, DestState, DestStateFips, DestStateName, DestWac, CRSDepTime, DepTime, DepDelay, DepDelayMinutes, DepDel15, DepartureDelayGroups, DepTimeBlk, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, ArrDelay, ArrDelayMinutes, ArrDel15, ArrivalDelayGroups, ArrTimeBlk, Cancelled, CancellationCode, Diverted, CRSElapsedTime, ActualElapsedTime, AirTime, Flights, Distance, DistanceGroup, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay, FirstDepTime, TotalAddGTime, LongestAddGTime, DivAirportLandi

+----+-------+-----+----------+---------+----------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+------+--------------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+---------------

60009778

In [122]:
from pyspark.sql.functions import col

# Step 1: Fill nulls in delay columns
delay_columns = ["NASDelay", "SecurityDelay", "LateAircraftDelay", "CarrierDelay", "WeatherDelay"]
df_filled = df.na.fill(0, subset=delay_columns)

# Step 2: Create TotalDelay and update original df
df = df_filled.withColumn(
    "TotalDelay",
    col("NASDelay") +
    col("SecurityDelay") +
    col("LateAircraftDelay") +
    col("CarrierDelay") +
    col("WeatherDelay")
)

# Step 3: Verify the update
df.show(n=20, truncate=False)


+----+-------+-----+----------+---------+----------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+------+------------------------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+-------------+---------+-------------+--------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+---

25/05/09 14:42:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Reporting_Airline, DOT_ID_Reporting_Airline, IATA_CODE_Reporting_Airline, Tail_Number, Flight_Number_Reporting_Airline, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, OriginWac, DestAirportID, DestAirportSeqID, DestCityMarketID, Dest, DestCityName, DestState, DestStateFips, DestStateName, DestWac, CRSDepTime, DepTime, DepDelay, DepDelayMinutes, DepDel15, DepartureDelayGroups, DepTimeBlk, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, ArrDelay, ArrDelayMinutes, ArrDel15, ArrivalDelayGroups, ArrTimeBlk, Cancelled, CancellationCode, Diverted, CRSElapsedTime, ActualElapsedTime, AirTime, Flights, Distance, DistanceGroup, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay, FirstDepTime, TotalAddGTime, LongestAddGTime, DivAirportLandi

In [123]:
from pyspark.sql.functions import col

weather_only_delays = df.filter(
    (col("TotalDelay") == col("WeatherDelay")) & (col("WeatherDelay") > 0)
)

# Show top 20 results with relevant columns
weather_only_delays.select("FlightDate", "Origin", "Dest", "DepDelay", "WeatherDelay","TotalDelay").show(20, truncate=False)
weather_only_delays.count()

+----------+------+----+--------+------------+----------+
|FlightDate|Origin|Dest|DepDelay|WeatherDelay|TotalDelay|
+----------+------+----+--------+------------+----------+
|2020-01-28|FLL   |JAX |125.0   |118.0       |118.0     |
|2020-01-08|PBI   |EWR |191.0   |176.0       |176.0     |
|2020-01-12|JFK   |BUF |276.0   |272.0       |272.0     |
|2020-01-12|BUF   |BOS |200.0   |187.0       |187.0     |
|2020-01-12|BUF   |FLL |203.0   |194.0       |194.0     |
|2020-01-14|SEA   |HNL |0.0     |43.0        |43.0      |
|2020-01-06|SEA   |HNL |110.0   |79.0        |79.0      |
|2020-01-13|SEA   |OGG |-2.0    |28.0        |28.0      |
|2020-01-14|SEA   |OGG |-2.0    |46.0        |46.0      |
|2020-01-11|CLT   |CMH |32.0    |20.0        |20.0      |
|2020-01-11|CLT   |GSP |26.0    |20.0        |20.0      |
|2020-01-19|DSM   |CLT |44.0    |36.0        |36.0      |
|2020-01-20|DSM   |CLT |29.0    |18.0        |18.0      |
|2020-01-04|HPN   |DCA |77.0    |76.0        |76.0      |
|2020-01-03|AV

223845

In [124]:
import pandas as pd

# Load airport info
airports_df = pd.read_csv("airports.csv")

# Filter valid IATA codes only
airports_df = airports_df[airports_df["iata_code"].notnull()]

# Build IATA → (lat, lon) mapping
iata_coords = dict(zip(
    airports_df["iata_code"],
    zip(airports_df["latitude_deg"], airports_df["longitude_deg"])
))

# Sample output
print("Example:", {k: iata_coords[k] for k in list(iata_coords)[:5]})


Example: {'UTK': (11.222219, 169.851429), 'OCA': (25.325399398804, -80.274803161621), 'CUX': (27.7211, -97.512802), 'CSE': (38.851918, -106.928341), 'CUS': (31.823898, -107.629924)}


In [12]:
# Extract all unique IATA airport codes involved in weather-delayed flights
from pyspark.sql.functions import col, to_date

# Convert date column
df = df.withColumn("FlightDate", to_date(col("FlightDate"), "yyyy-MM-dd"))

# Extract unique IATA codes
airport_codes = weather_only_delays.select("Origin").union(weather_only_delays.select("Dest")) \
    .distinct().rdd.flatMap(lambda x: x).collect()

print("Airports with weather delays:", airport_codes)


Airports with weather delays: ['BGM', 'SNA', 'BUR', 'GRB', 'GRR', 'PVU', 'EUG', 'PVD', 'GSO', 'OAK', 'MQT', 'MSN', 'FAR', 'BTM', 'ESC', 'RFD', 'DCA', 'GTR', 'MLU', 'CIU', 'LEX', 'RDM', 'JMS', 'ORF', 'CRW', 'CDV', 'TRI', 'CMH', 'LBF', 'TYR', 'CAK', 'MOB', 'IAH', 'HNL', 'SLN', 'SHV', 'CVG', 'BUF', 'LGA', 'HPN', 'RDD', 'AUS', 'MLI', 'ATW', 'AVL', 'LYH', 'LGB', 'CNY', 'SRQ', 'RNO', 'TTN', 'JAC', 'CHS', 'TUL', 'MSY', 'GEG', 'IDA', 'ATY', 'FSM', 'PIB', 'CID', 'SWO', 'EVV', 'OGS', 'GCK', 'SAV', 'CHO', 'PNS', 'SJC', 'TLH', 'ACT', 'DHN', 'GFK', 'SBN', 'EYW', 'RST', 'RSW', 'HRL', 'GJT', 'INL', 'GTF', 'JLN', 'EAR', 'COD', 'LWS', 'HLN', 'KTN', 'DIK', 'GCC', 'SJU', 'RIW', 'PSG', 'WRG', 'SCE', 'VCT', 'CDC', 'BFL', 'BJI', 'LIH', 'ERI', 'LWB', 'MYR', 'CDB', 'CWA', 'TOL', 'DLG', 'FOD', 'IAG', 'DDC', 'HGR', 'DRT', 'FLO', 'TBN', 'SCC', 'PPG', 'ADK', 'GST', 'SPN', 'PSE', 'BOS', 'MLB', 'EWR', 'LAS', 'JAN', 'IMT', 'DLH', 'DEN', 'RHI', 'ALB', 'IAD', 'SBA', 'BRO', 'SEA', 'CMI', 'LAN', 'VEL', 'PBG', 'VLD', 'MC

In [125]:
from meteostat import Stations, Hourly
from datetime import datetime
import pandas as pd

start = datetime(2020, 1, 1)
end = datetime(2024, 12, 31)

weather_dataframes = []

def fetch_full_hourly_weather(iata_code, start, end):
    if iata_code not in iata_coords:
        return pd.DataFrame()

    lat, lon = iata_coords[iata_code]
    stations = Stations().nearby(lat, lon).fetch(5)

    for i in range(len(stations)):
        station_id = stations.index[i]
        h_start = pd.to_datetime(stations.iloc[i]['hourly_start'])
        h_end = pd.to_datetime(stations.iloc[i]['hourly_end'])

        if start >= h_start and end <= h_end:
            try:
                df = Hourly(station_id, start, end).fetch()
                if df.empty:
                    continue
                df = df.reset_index()
                df["Airport"] = iata_code
                df.rename(columns={"time": "WeatherDate"}, inplace=True)
                return df[["WeatherDate", "Airport", "temp", "wspd", "prcp", "rhum"]]
            except:
                continue
    return pd.DataFrame()

# Now use it like before
for code in airport_codes:
    print(f"📡 Fetching weather for {code}...")
    wdf = fetch_full_hourly_weather(code, start, end)
    if not wdf.empty:
        weather_dataframes.append(wdf)
        print(f"{code}: {len(wdf)} records")
    else:
        print(f"⚠️ No data for {code}")


📡 Fetching weather for BGM...
BGM: 43796 records
📡 Fetching weather for SNA...
SNA: 43822 records
📡 Fetching weather for BUR...
BUR: 43825 records
📡 Fetching weather for GRB...
GRB: 43825 records
📡 Fetching weather for GRR...
GRR: 43823 records
📡 Fetching weather for PVU...
PVU: 43793 records
📡 Fetching weather for EUG...
EUG: 43825 records
📡 Fetching weather for PVD...
PVD: 43821 records
📡 Fetching weather for GSO...
GSO: 43825 records
📡 Fetching weather for OAK...
OAK: 43825 records
📡 Fetching weather for MQT...
MQT: 43594 records
📡 Fetching weather for MSN...
MSN: 43825 records
📡 Fetching weather for FAR...
FAR: 43825 records
📡 Fetching weather for BTM...
BTM: 43458 records
📡 Fetching weather for ESC...
ESC: 43745 records
📡 Fetching weather for RFD...
RFD: 43825 records
📡 Fetching weather for DCA...
DCA: 43825 records
📡 Fetching weather for GTR...
GTR: 43753 records
📡 Fetching weather for MLU...
MLU: 43822 records
📡 Fetching weather for CIU...
CIU: 43553 records
📡 Fetching weather f

MMH: 43542 records
📡 Fetching weather for LSE...
LSE: 43825 records
📡 Fetching weather for FAY...
FAY: 43823 records
📡 Fetching weather for AMA...
AMA: 43825 records
📡 Fetching weather for MAF...
MAF: 43825 records
📡 Fetching weather for BIS...
BIS: 43825 records
📡 Fetching weather for FAI...
FAI: 43825 records
📡 Fetching weather for XNA...
XNA: 43823 records
📡 Fetching weather for SGU...
SGU: 43820 records
📡 Fetching weather for LNK...
LNK: 43825 records
📡 Fetching weather for PSP...
PSP: 43822 records
📡 Fetching weather for BOI...
BOI: 43825 records
📡 Fetching weather for MEI...
MEI: 43822 records
📡 Fetching weather for PIR...
PIR: 43074 records
📡 Fetching weather for FLG...
FLG: 43770 records
📡 Fetching weather for TVC...
TVC: 43824 records
📡 Fetching weather for ORH...
ORH: 43424 records
📡 Fetching weather for ACV...
ACV: 43824 records
📡 Fetching weather for MRY...
MRY: 36205 records
📡 Fetching weather for BDL...
BDL: 43825 records
📡 Fetching weather for CLE...
CLE: 43825 records
📡

MSO: 43825 records
📡 Fetching weather for STT...
STT: 43812 records
📡 Fetching weather for FNT...
FNT: 43775 records
📡 Fetching weather for DEC...
DEC: 43805 records
📡 Fetching weather for GUM...
GUM: 43813 records
📡 Fetching weather for YAK...
YAK: 43825 records
📡 Fetching weather for OTZ...
OTZ: 43756 records
📡 Fetching weather for GGG...
GGG: 43822 records
📡 Fetching weather for BQK...
BQK: 43816 records
📡 Fetching weather for BQN...
BQN: 41025 records
📡 Fetching weather for PHF...
PHF: 43824 records
📡 Fetching weather for LCH...
LCH: 42276 records
📡 Fetching weather for LBE...
LBE: 43476 records
📡 Fetching weather for OAJ...
OAJ: 43783 records
📡 Fetching weather for SIT...
SIT: 43804 records
📡 Fetching weather for WYS...
WYS: 42409 records
📡 Fetching weather for SJT...
SJT: 43819 records
📡 Fetching weather for STS...
STS: 43821 records
📡 Fetching weather for HTS...
HTS: 43823 records
📡 Fetching weather for AKN...
AKN: 43806 records
📡 Fetching weather for EAT...
EAT: 43822 records
📡

In [74]:
df = df.withColumn("FlightDate", to_date(col("FlightDate"), "yyyy-MM-dd"))

In [75]:
weather_df = pd.concat(weather_dataframes, ignore_index=True)
weather_df

,WeatherDate,Airport,temp,wspd,prcp,rhum
0,2020-01-01 01:00:00,BGM,0.0,24.1,NaN,66.0
1,2020-01-01 02:00:00,BGM,0.0,16.6,NaN,66.0
2,2020-01-01 03:00:00,BGM,-0.6,20.5,NaN,72.0
3,2020-01-01 04:00:00,BGM,-0.6,18.4,NaN,69.0
4,2020-01-01 05:00:00,BGM,-0.6,24.1,NaN,69.0
...,...,...,...,...,...,...
16267821,2024-12-30 20:00:00,MGW,11.1,14.8,0.0,41.0
16267822,2024-12-30 21:00:00,MGW,11.1,14.8,0.0,41.0
16267823,2024-12-30 22:00:00,MGW,9.4,11.2,0.0,46.0
16267824,2024-12-30 23:00:00,MGW,8.3,9.4,0.0,49.0


In [76]:
weather_df["Date"] = pd.to_datetime(weather_df["WeatherDate"]).dt.date
weather_df["Hour"] = pd.to_datetime(weather_df["WeatherDate"]).dt.hour
weather_df

,WeatherDate,Airport,temp,wspd,prcp,rhum,Date,Hour
0,2020-01-01 01:00:00,BGM,0.0,24.1,NaN,66.0,2020-01-01,1
1,2020-01-01 02:00:00,BGM,0.0,16.6,NaN,66.0,2020-01-01,2
2,2020-01-01 03:00:00,BGM,-0.6,20.5,NaN,72.0,2020-01-01,3
3,2020-01-01 04:00:00,BGM,-0.6,18.4,NaN,69.0,2020-01-01,4
4,2020-01-01 05:00:00,BGM,-0.6,24.1,NaN,69.0,2020-01-01,5
...,...,...,...,...,...,...,...,...
16267821,2024-12-30 20:00:00,MGW,11.1,14.8,0.0,41.0,2024-12-30,20
16267822,2024-12-30 21:00:00,MGW,11.1,14.8,0.0,41.0,2024-12-30,21
16267823,2024-12-30 22:00:00,MGW,9.4,11.2,0.0,46.0,2024-12-30,22
16267824,2024-12-30 23:00:00,MGW,8.3,9.4,0.0,49.0,2024-12-30,23


In [126]:
flight_sample = weather_only_delays.select(
    "FlightDate", "Origin", "Dest", "DepTime", "DepDelay", "ArrDelay", "WeatherDelay",
    "TotalDelay", "ArrTime"
).withColumn("DepHour", floor(col("DepTime") / 100)).toPandas()
flight_sample["Date"] = pd.to_datetime(flight_sample["FlightDate"]).dt.date
flight_sample

,FlightDate,Origin,Dest,DepTime,DepDelay,ArrDelay,WeatherDelay,TotalDelay,ArrTime,DepHour,Date
0,2020-01-28,FLL,JAX,1205,125.0,118.0,118.0,118.0,1317,12,2020-01-28
1,2020-01-08,PBI,EWR,1754,191.0,176.0,176.0,176.0,2040,17,2020-01-08
2,2020-01-12,JFK,BUF,1106,276.0,272.0,272.0,272.0,1226,11,2020-01-12
3,2020-01-12,BUF,BOS,956,200.0,187.0,187.0,187.0,1112,9,2020-01-12
4,2020-01-12,BUF,FLL,903,203.0,194.0,194.0,194.0,1213,9,2020-01-12
...,...,...,...,...,...,...,...,...,...,...,...
223840,2023-10-05,BOS,PIT,951,66.0,47.0,47.0,47.0,1126,9,2023-10-05
223841,2023-10-05,LGA,BOS,849,49.0,23.0,23.0,23.0,947,8,2023-10-05
223842,2022-07-23,ATL,BOS,1733,78.0,70.0,70.0,70.0,2009,17,2022-07-23
223843,2022-07-23,ATL,RDU,2332,58.0,52.0,52.0,52.0,51,23,2022-07-23


In [128]:
origin_weather_df = weather_df.rename(columns={
    "Airport": "Origin",
    "temp": "Origin_Temp",
    "wspd": "Origin_Wind",
    "prcp": "Origin_Precip",
    "rhum": "Origin_Humidity",
    "Hour": "DepHour"
})

merged_df = pd.merge(
    flight_sample,
    origin_weather_df,
    how="left",
    on=["Origin", "Date", "DepHour"]
)
merged_df

,FlightDate,Origin,Dest,DepTime,DepDelay,ArrDelay,WeatherDelay,TotalDelay,ArrTime,DepHour,Date,WeatherDate,Origin_Temp,Origin_Wind,Origin_Precip,Origin_Humidity
0,2020-01-28,FLL,JAX,1205,125.0,118.0,118.0,118.0,1317,12,2020-01-28,2020-01-28 12:00:00,17.2,7.6,0.0,90.0
1,2020-01-08,PBI,EWR,1754,191.0,176.0,176.0,176.0,2040,17,2020-01-08,2020-01-08 17:00:00,24.4,14.8,0.0,50.0
2,2020-01-12,JFK,BUF,1106,276.0,272.0,272.0,272.0,1226,11,2020-01-12,2020-01-12 11:00:00,12.2,40.7,NaN,100.0
3,2020-01-12,BUF,BOS,956,200.0,187.0,187.0,187.0,1112,9,2020-01-12,2020-01-12 09:00:00,6.1,13.0,NaN,93.0
4,2020-01-12,BUF,FLL,903,203.0,194.0,194.0,194.0,1213,9,2020-01-12,2020-01-12 09:00:00,6.1,13.0,NaN,93.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223840,2023-10-05,BOS,PIT,951,66.0,47.0,47.0,47.0,1126,9,2023-10-05,2023-10-05 09:00:00,16.1,0.0,0.0,97.0
223841,2023-10-05,LGA,BOS,849,49.0,23.0,23.0,23.0,947,8,2023-10-05,2023-10-05 08:00:00,17.2,0.0,0.0,84.0
223842,2022-07-23,ATL,BOS,1733,78.0,70.0,70.0,70.0,2009,17,2022-07-23,2022-07-23 17:00:00,30.0,0.0,0.0,65.0
223843,2022-07-23,ATL,RDU,2332,58.0,52.0,52.0,52.0,51,23,2022-07-23,2022-07-23 23:00:00,29.4,7.6,0.0,63.0


In [129]:
# Drop any conflicting destination weather columns from previous merges
merged_df = merged_df.drop(columns=[
    'Dest_Temp_x', 'Dest_Wind_x', 'Dest_Precip_x', 'Dest_Humidity_x',
    'Dest_Temp_y', 'Dest_Wind_y', 'Dest_Precip_y', 'Dest_Humidity_y',
    'WeatherDate_y'
], errors='ignore')


In [130]:
merged_df["ArrHour"] = (merged_df["ArrTime"] // 100).astype("Int64")

In [131]:

dest_weather_df = weather_df.rename(columns={
    "Airport": "Dest",
    "temp": "Dest_Temp",
    "wspd": "Dest_Wind",
    "prcp": "Dest_Precip",
    "rhum": "Dest_Humidity",
    "Hour": "ArrHour"
})

merged_df = pd.merge(
    merged_df,
    dest_weather_df,
    how="left",
    on=["Dest", "Date", "ArrHour"]
)
merged_df

,FlightDate,Origin,Dest,DepTime,DepDelay,ArrDelay,WeatherDelay,TotalDelay,ArrTime,DepHour,...,Origin_Temp,Origin_Wind,Origin_Precip,Origin_Humidity,ArrHour,WeatherDate_y,Dest_Temp,Dest_Wind,Dest_Precip,Dest_Humidity
0,2020-01-28,FLL,JAX,1205,125.0,118.0,118.0,118.0,1317,12,...,17.2,7.6,0.0,90.0,13,2020-01-28 13:00:00,8.9,11.2,0.0,86.0
1,2020-01-08,PBI,EWR,1754,191.0,176.0,176.0,176.0,2040,17,...,24.4,14.8,0.0,50.0,20,2020-01-08 20:00:00,5.1,48.2,NaN,30.0
2,2020-01-12,JFK,BUF,1106,276.0,272.0,272.0,272.0,1226,11,...,12.2,40.7,NaN,100.0,12,2020-01-12 12:00:00,1.5,38.9,NaN,96.0
3,2020-01-12,BUF,BOS,956,200.0,187.0,187.0,187.0,1112,9,...,6.1,13.0,NaN,93.0,11,2020-01-12 11:00:00,18.3,42.5,NaN,78.0
4,2020-01-12,BUF,FLL,903,203.0,194.0,194.0,194.0,1213,9,...,6.1,13.0,NaN,93.0,12,2020-01-12 12:00:00,24.4,27.7,0.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223840,2023-10-05,BOS,PIT,951,66.0,47.0,47.0,47.0,1126,9,...,16.1,0.0,0.0,97.0,11,2023-10-05 11:00:00,16.1,0.0,0.0,81.0
223841,2023-10-05,LGA,BOS,849,49.0,23.0,23.0,23.0,947,8,...,17.2,0.0,0.0,84.0,9,2023-10-05 09:00:00,16.1,0.0,0.0,97.0
223842,2022-07-23,ATL,BOS,1733,78.0,70.0,70.0,70.0,2009,17,...,30.0,0.0,0.0,65.0,20,2022-07-23 20:00:00,31.7,16.6,0.0,37.0
223843,2022-07-23,ATL,RDU,2332,58.0,52.0,52.0,52.0,51,23,...,29.4,7.6,0.0,63.0,0,2022-07-23 00:00:00,28.3,0.0,0.0,72.0


In [96]:
# Step 1: Drop duplicate destination weather columns if they exist
columns_to_drop = [
    "Dest_Temp_x", "Dest_Wind_x", "Dest_Precip_x", "Dest_Humidity_x", "Dest_WeatherDate_y", "WeatherDate_y", "Dest_WeatherDate.1"
]

merged_df = merged_df.drop(
    columns=[col for col in columns_to_drop if col in merged_df.columns]
)

# Step 2: Rename destination weather columns to final names
rename_cols = {
    "Dest_Temp_y": "Dest_Temp",
    "Dest_Wind_y": "Dest_Wind",
    "Dest_Precip_y": "Dest_Precip",
    "Dest_Humidity_y": "Dest_Humidity",
    "WeatherDate": "Dest_WeatherDate"
}

merged_df = merged_df.rename(columns={k: v for k, v in rename_cols.items() if k in merged_df.columns})
merged_df



,FlightDate,Origin,Dest,DepTime,DepDelay,ArrDelay,WeatherDelay,TotalDelay,ArrTime,DepHour,...,Origin_Wind,Origin_Precip,Origin_Humidity,ArrHour,Dest_WeatherDate,Dest_WeatherDate,Dest_Temp,Dest_Wind,Dest_Precip,Dest_Humidity
0,2020-01-28,FLL,JAX,1205,125.0,118.0,118.0,118.0,1317,12,...,7.6,0.0,90.0,13,2020-01-28 13:00:00,2020-01-28 13:00:00,8.9,11.2,0.0,86.0
1,2020-01-08,PBI,EWR,1754,191.0,176.0,176.0,176.0,2040,17,...,14.8,0.0,50.0,20,2020-01-08 20:00:00,2020-01-08 20:00:00,5.1,48.2,NaN,30.0
2,2020-01-12,JFK,BUF,1106,276.0,272.0,272.0,272.0,1226,11,...,40.7,NaN,100.0,12,2020-01-12 12:00:00,2020-01-12 12:00:00,1.5,38.9,NaN,96.0
3,2020-01-12,BUF,BOS,956,200.0,187.0,187.0,187.0,1112,9,...,13.0,NaN,93.0,11,2020-01-12 11:00:00,2020-01-12 11:00:00,18.3,42.5,NaN,78.0
4,2020-01-12,BUF,FLL,903,203.0,194.0,194.0,194.0,1213,9,...,13.0,NaN,93.0,12,2020-01-12 12:00:00,2020-01-12 12:00:00,24.4,27.7,0.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223840,2023-10-05,BOS,PIT,951,66.0,47.0,47.0,47.0,1126,9,...,0.0,0.0,97.0,11,2023-10-05 11:00:00,2023-10-05 11:00:00,16.1,0.0,0.0,81.0
223841,2023-10-05,LGA,BOS,849,49.0,23.0,23.0,23.0,947,8,...,0.0,0.0,84.0,9,2023-10-05 09:00:00,2023-10-05 09:00:00,16.1,0.0,0.0,97.0
223842,2022-07-23,ATL,BOS,1733,78.0,70.0,70.0,70.0,2009,17,...,0.0,0.0,65.0,20,2022-07-23 20:00:00,2022-07-23 20:00:00,31.7,16.6,0.0,37.0
223843,2022-07-23,ATL,RDU,2332,58.0,52.0,52.0,52.0,51,23,...,7.6,0.0,63.0,0,2022-07-23 00:00:00,2022-07-23 00:00:00,28.3,0.0,0.0,72.0


In [132]:
# Step 1: Print all columns with their index
for idx, col in enumerate(merged_df.columns):
    print(f"{idx}: {col}")


0: FlightDate
1: Origin
2: Dest
3: DepTime
4: DepDelay
5: ArrDelay
6: WeatherDelay
7: TotalDelay
8: ArrTime
9: DepHour
10: Date
11: WeatherDate_x
12: Origin_Temp
13: Origin_Wind
14: Origin_Precip
15: Origin_Humidity
16: ArrHour
17: WeatherDate_y
18: Dest_Temp
19: Dest_Wind
20: Dest_Precip
21: Dest_Humidity


In [133]:
def classify_weather_origin(row):
    if pd.isna(row["Origin_Temp"]) or pd.isna(row["Origin_Wind"]):
        return "Unknown"

    # Strict 'Clear' comes FIRST
    if (
        10 <= row["Origin_Temp"] <= 25 and
        row["Origin_Wind"] < 10 and
        row["Origin_Precip"] == 0.0 and
        row["Origin_Humidity"] < 70
    ):
        return "Clear"

    # Other conditions
    if row["Origin_Temp"] < 5:
        return "Cold"
    if row["Origin_Temp"] > 30:
        return "Hot"
    if row["Origin_Wind"] > 25:
        return "Windy"
    if row["Origin_Precip"] >= 2 or row["Origin_Humidity"] > 90:
        return "Stormy"

    return "Other"

def classify_weather_dest(row):
    if pd.isna(row["Dest_Temp"]) or pd.isna(row["Dest_Wind"]):
        return "Unknown"

    # Strict 'Clear' comes FIRST
    if (
        10 <= row["Dest_Temp"] <= 25 and
        row["Dest_Wind"] < 10 and
        row["Dest_Precip"] == 0.0 and
        row["Dest_Humidity"] < 70
    ):
        return "Clear"

    # Other conditions
    if row["Dest_Temp"] < 5:
        return "Cold"
    if row["Dest_Temp"] > 30:
        return "Hot"
    if row["Dest_Wind"] > 25:
        return "Windy"
    if row["Dest_Precip"] >= 2 or row["Dest_Humidity"] > 90:
        return "Stormy"

    return "Other"



In [135]:
# Step 2: Drop by index if duplicate column names exist
if merged_df.columns.duplicated().any():
    merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]


In [136]:
merged_df

,FlightDate,Origin,Dest,DepTime,DepDelay,ArrDelay,WeatherDelay,TotalDelay,ArrTime,DepHour,...,Origin_Temp,Origin_Wind,Origin_Precip,Origin_Humidity,ArrHour,WeatherDate_y,Dest_Temp,Dest_Wind,Dest_Precip,Dest_Humidity
0,2020-01-28,FLL,JAX,1205,125.0,118.0,118.0,118.0,1317,12,...,17.2,7.6,0.0,90.0,13,2020-01-28 13:00:00,8.9,11.2,0.0,86.0
1,2020-01-08,PBI,EWR,1754,191.0,176.0,176.0,176.0,2040,17,...,24.4,14.8,0.0,50.0,20,2020-01-08 20:00:00,5.1,48.2,NaN,30.0
2,2020-01-12,JFK,BUF,1106,276.0,272.0,272.0,272.0,1226,11,...,12.2,40.7,NaN,100.0,12,2020-01-12 12:00:00,1.5,38.9,NaN,96.0
3,2020-01-12,BUF,BOS,956,200.0,187.0,187.0,187.0,1112,9,...,6.1,13.0,NaN,93.0,11,2020-01-12 11:00:00,18.3,42.5,NaN,78.0
4,2020-01-12,BUF,FLL,903,203.0,194.0,194.0,194.0,1213,9,...,6.1,13.0,NaN,93.0,12,2020-01-12 12:00:00,24.4,27.7,0.0,88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223840,2023-10-05,BOS,PIT,951,66.0,47.0,47.0,47.0,1126,9,...,16.1,0.0,0.0,97.0,11,2023-10-05 11:00:00,16.1,0.0,0.0,81.0
223841,2023-10-05,LGA,BOS,849,49.0,23.0,23.0,23.0,947,8,...,17.2,0.0,0.0,84.0,9,2023-10-05 09:00:00,16.1,0.0,0.0,97.0
223842,2022-07-23,ATL,BOS,1733,78.0,70.0,70.0,70.0,2009,17,...,30.0,0.0,0.0,65.0,20,2022-07-23 20:00:00,31.7,16.6,0.0,37.0
223843,2022-07-23,ATL,RDU,2332,58.0,52.0,52.0,52.0,51,23,...,29.4,7.6,0.0,63.0,0,2022-07-23 00:00:00,28.3,0.0,0.0,72.0


In [137]:
# Apply weather classification functions to the merged dataframe
merged_df["Origin_WeatherCondition"] = merged_df.apply(classify_weather_origin, axis=1)
merged_df["Dest_WeatherCondition"] = merged_df.apply(classify_weather_dest, axis=1)


In [138]:
merged_df[[
    "FlightDate", "Origin", "Origin_Temp", "Origin_Wind","Origin_Precip", "Origin_WeatherCondition",
    "Dest", "Dest_Temp", "Dest_Wind","Dest_Precip", "Dest_WeatherCondition"
]].head(10)


,FlightDate,Origin,Origin_Temp,Origin_Wind,Origin_Precip,Origin_WeatherCondition,Dest,Dest_Temp,Dest_Wind,Dest_Precip,Dest_WeatherCondition
0,2020-01-28,FLL,17.2,7.6,0.0,Other,JAX,8.9,11.2,0.0,Other
1,2020-01-08,PBI,24.4,14.8,0.0,Other,EWR,5.1,48.2,NaN,Windy
2,2020-01-12,JFK,12.2,40.7,NaN,Windy,BUF,1.5,38.9,NaN,Cold
3,2020-01-12,BUF,6.1,13.0,NaN,Stormy,BOS,18.3,42.5,NaN,Windy
4,2020-01-12,BUF,6.1,13.0,NaN,Stormy,FLL,24.4,27.7,0.0,Windy
5,2020-01-14,SEA,-1.1,16.6,NaN,Cold,HNL,25.0,25.9,0.0,Windy
6,2020-01-06,SEA,5.0,16.6,0.8,Other,HNL,23.3,18.4,0.0,Other
7,2020-01-13,SEA,1.1,20.5,0.0,Cold,OGG,22.8,11.2,0.0,Other
8,2020-01-14,SEA,-1.7,14.8,NaN,Cold,OGG,21.1,7.6,0.0,Other
9,2020-01-11,CLT,20.6,35.3,NaN,Windy,CMH,12.8,14.8,NaN,Other


In [139]:
# Drop the old column if it still exists
if "Origin_WeatherCondition" in merged_df.columns:
    merged_df = merged_df.drop(columns=["Origin_WeatherCondition"])

# Apply the updated classifier freshly
merged_df["Origin_WeatherCondition"] = merged_df.apply(classify_weather_origin, axis=1)


In [140]:
print(
    merged_df[merged_df["WeatherDelay"] > 0]
    .groupby("Origin_WeatherCondition")
    .size()
    .reset_index(name="DelayedFlights")
)


  Origin_WeatherCondition  DelayedFlights
0                   Clear            9903
1                    Cold           47673
2                     Hot           34421
3                   Other           91765
4                  Stormy           24617
5                 Unknown             627
6                   Windy           14839


In [141]:
print(
    merged_df[merged_df["WeatherDelay"] > 0]
    .groupby("Dest_WeatherCondition")
    .size()
    .reset_index(name="DelayedFlights")
)


  Dest_WeatherCondition  DelayedFlights
0                 Clear           12675
1                  Cold           42573
2                   Hot           31000
3                 Other           99813
4                Stormy           20326
5               Unknown             893
6                 Windy           16565


In [142]:
filtered_df = merged_df[
    (~merged_df["Origin_WeatherCondition"].isin(["Unknown", "Other"])) &
    (~merged_df["Dest_WeatherCondition"].isin(["Unknown", "Other"]))
].copy()


In [143]:
filtered_df 


,FlightDate,Origin,Dest,DepTime,DepDelay,ArrDelay,WeatherDelay,TotalDelay,ArrTime,DepHour,...,Origin_Precip,Origin_Humidity,ArrHour,WeatherDate_y,Dest_Temp,Dest_Wind,Dest_Precip,Dest_Humidity,Dest_WeatherCondition,Origin_WeatherCondition
2,2020-01-12,JFK,BUF,1106,276.0,272.0,272.0,272.0,1226,11,...,NaN,100.0,12,2020-01-12 12:00:00,1.5,38.9,NaN,96.0,Cold,Windy
3,2020-01-12,BUF,BOS,956,200.0,187.0,187.0,187.0,1112,9,...,NaN,93.0,11,2020-01-12 11:00:00,18.3,42.5,NaN,78.0,Windy,Stormy
4,2020-01-12,BUF,FLL,903,203.0,194.0,194.0,194.0,1213,9,...,NaN,93.0,12,2020-01-12 12:00:00,24.4,27.7,0.0,88.0,Windy,Stormy
5,2020-01-14,SEA,HNL,835,0.0,43.0,43.0,43.0,1338,8,...,NaN,88.0,13,2020-01-14 13:00:00,25.0,25.9,0.0,66.0,Windy,Cold
12,2020-01-20,DSM,CLT,719,29.0,18.0,18.0,18.0,1046,7,...,NaN,76.0,10,2020-01-20 10:00:00,-2.2,14.8,0.0,55.0,Cold,Cold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223781,2023-07-09,EWR,IAH,1957,478.0,473.0,473.0,473.0,2246,19,...,0.6,91.0,22,2023-07-09 22:00:00,33.3,9.4,0.0,60.0,Hot,Stormy
223804,2023-07-06,IAD,MCO,1406,71.0,51.0,51.0,51.0,1607,14,...,0.0,61.0,16,2023-07-06 16:00:00,32.8,5.4,0.0,60.0,Hot,Hot
223807,2023-07-06,IAD,DFW,1401,68.0,65.0,65.0,65.0,1602,14,...,0.0,61.0,16,2023-07-06 16:00:00,32.2,0.0,0.0,58.0,Hot,Hot
223813,2023-08-25,HPN,BOS,1324,29.0,29.0,29.0,29.0,1428,13,...,1.8,94.0,14,2023-08-25 14:00:00,19.4,22.3,13.0,93.0,Stormy,Stormy


In [144]:
print("Origin Weather Condition Counts:")
print(
    filtered_df.groupby("Origin_WeatherCondition")
    .size()
    .reset_index(name="DelayedFlights")
)


Origin Weather Condition Counts:
  Origin_WeatherCondition  DelayedFlights
0                   Clear            4663
1                    Cold           36674
2                     Hot           18672
3                  Stormy           12153
4                   Windy            8025


In [145]:
print("Destination Weather Condition Counts:")
print(
    filtered_df.groupby("Dest_WeatherCondition")
    .size()
    .reset_index(name="DelayedFlights")
)

Destination Weather Condition Counts:
  Dest_WeatherCondition  DelayedFlights
0                 Clear            6096
1                  Cold           35342
2                   Hot           18179
3                Stormy           11085
4                 Windy            9485
